## Part A: Data Collection 

RB Player Data is getting web scraped from ProFootballReference site between the years 2014-2024

In [ ]:
import scrapy

class PlayersSpider(scrapy.Spider):
    name = "rb_rushing"
    allowedDomain = ["pro-football-reference.com"]

    def start_requests(self):
        for year in range (2014, 2024):
            url1 = f"https://www.pro-football-reference.com/years/{year}/rushing.htm"
            yield scrapy.Request(
                url=url1,
                callback=self.parse_rushing,
                meta={"year": year}
            )

            if year > 2017:
                url2 = f"https://www.pro-football-reference.com/years/{year}/rushing_advanced.htm"
                yield scrapy.Request(
                    url=url2,
                    callback=self.parse_advanced_rushing,
                    meta={"year": year}
                )
    def parse_rushing(self, response):
        year=response.meta['year']
        for row in response.css("table#rushing tbody tr"):
            player=row.css("td[data-stat='player'] a::text").get()
            if not player:
                continue
            yield {
                "year": year,
                "player": player,
                "age": row.css("td[data-stat='age'] a::text").get(),
                "team": row.css("td[data-stat='team'] a::text").get(),
                "games": row.css("td[data-stat='games'] ::text").get(),
                "rush_att": row.css("td[data-stat='rush_att']::text").get(),
                "rush_yds": row.css("td[data-stat='rush_yds']::text").get(),
                "rush_td": row.css("td[data-stat='rush_td']::text").get(),
                "rush_success": row.css("td[data-stat='rush_success']::text").get(),


                }